In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from utils.tools import dotdict
import matplotlib.pyplot as plt
from exp.exp_informer import Exp_Informer
import torch
import numpy as np
%matplotlib 
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'


Using matplotlib backend: Qt5Agg


In [7]:
args = dotdict()
 
 
args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]
args.data = 'custom' # data
args.root_path = './data/CustomDataSet/' # root path of data file
args.data_path = 'dust.csv' # data file
args.features = 'S' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'value' # target feature in S or MS task
args.freq = 't' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './checkpoints' # location of model checkpoints
 
 # 96 48 48
args.seq_len = 64 # input sequence length of Informer encoder
args.label_len = 32 # start token length of Informer decoder
args.pred_len = 32 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]
 
# 4 4 1
#1 1 1
args.enc_in = 1 # encoder input size
args.dec_in = 1 # decoder input size
args.c_out = 1 # output size
args.factor = 5 # probsparse attn factor
args.padding = 0  # padding type
args.d_model = 256 # dimension of model
args.n_heads = 4 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 256 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
 
 
args.batch_size = 32
args.learning_rate = 0.00001
args.loss = 'rmse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training
args.inverse=False
 
 
args.num_workers = 0
args.train_epochs = 10000
args.patience = 3
args.des = 'exp'
 
 
# args.use_gpu = True if torch.cuda.is_available() else False
args.use_gpu = True
args.gpu = 0
 
 
args.use_multi_gpu = False
args.devices = '0,1,2,3'

args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [8]:
Exp = Exp_Informer

# setting record of experiments
setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_{}'.format(args.data_path.split('.')[0], args.model, args.data, args.features,
            args.seq_len, args.label_len, args.pred_len,
            args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.des)
 
 
# set experiments
exp = Exp(args)
print(args)
 
# 
# # train
# print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
# exp.train(setting)
# 
# 
# # test
# print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
# exp.test(setting)

# 
exp.predict(setting, True)

Use GPU: cuda:0
{'model': 'informer', 'data': 'custom', 'root_path': './data/CustomDataSet/', 'data_path': 'dust.csv', 'features': 'S', 'target': 'value', 'freq': 't', 'checkpoints': './checkpoints', 'seq_len': 64, 'label_len': 32, 'pred_len': 32, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'factor': 5, 'padding': 0, 'd_model': 256, 'n_heads': 4, 'e_layers': 2, 'd_layers': 1, 'd_ff': 256, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'batch_size': 32, 'learning_rate': 1e-05, 'loss': 'rmse', 'lradj': 'type1', 'use_amp': False, 'inverse': False, 'num_workers': 0, 'train_epochs': 10000, 'patience': 3, 'des': 'exp', 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 't'}
pred 1


In [11]:
exp.args.inverse=True
exp.predict(setting,True)

pred 1


In [12]:
#预测的结果
prediction = np.load('./results/' + setting + '/real_prediction.npy')
plt.figure()
plt.plot(prediction[0, :, -1])
plt.show()

In [13]:
prediction.size

48

In [42]:
prediction[0,:,-1]

array([0.00167099, 0.00172254, 0.001768  , 0.00181342, 0.00185989,
       0.00202564, 0.00206059, 0.00209383, 0.00212466, 0.00215241,
       0.00217629, 0.00219561, 0.00221039, 0.00222164, 0.00223102,
       0.00224017, 0.00225007, 0.00226088, 0.00227203, 0.00228255,
       0.00229147, 0.002299  , 0.00230365, 0.00230625, 0.0023079 ,
       0.0023098 , 0.00231277, 0.0023169 , 0.00232153, 0.00232579,
       0.00230211, 0.0023039 , 0.00230563, 0.00230768, 0.00230993,
       0.00231206, 0.00231391, 0.00231573, 0.00231805, 0.00232125,
       0.00232517, 0.00232919, 0.00233257, 0.00233496, 0.00233654,
       0.00233788, 0.00233956, 0.00232753], dtype=float32)

In [17]:
setting='informer_custom_ftMS_sl96_ll48_pl48_dm256_nh4_el2_dl1_df256_atprob_fc5_ebtimeF_dtTrue_exp'

In [14]:
test_loss_plot_data = np.load('./results/' +setting + '/test_loss_every_epoch.npy')
train_loss_plot_data = np.load('./results/' + setting + '/train_loss_every_epoch.npy')
vali_loss_plot_data = np.load('./results/' + setting + '/vali_loss_every_epoch.npy')
# test_loss_every_epoch.shape

plt.figure()
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.xlabel('Epoch')
plt.ylabel('RMSE')

plt.plot(test_loss_plot_data[:], label='test')
plt.plot(train_loss_plot_data[:], label='training')
plt.plot(vali_loss_plot_data[:], label='validation')
plt.legend(frameon=False)

plt.show()

In [78]:
test_loss_plot_data

array([0.21206684, 0.1760576 , 0.15392497, 0.16048865, 0.14486709,
       0.13524537, 0.13098001, 0.13288058, 0.12932946, 0.13219103],
      dtype=float32)

In [3]:
t = np.load('./results/' + "informer_custom_ftMS_sl96_ll48_pl48_dm256_nh4_el2_dl1_df256_atprob_fc5_ebtimeF_dtTrue_exp" + '/train_loss_every_epoch.npy')
t

array([0.15907846, 0.08363054, 0.06544833, 0.05128703, 0.04483822,
       0.03735077])

In [15]:
#测试集上 真值和预测值的比较
preds = np.load('./results/' + setting + '/pred.npy')
trues = np.load('./results/' + setting + '/true.npy')


plt.figure()
plt.plot(trues[140, :, -1], label='GroundTruth')
plt.plot(preds[140, :, -1], label='Prediction')
plt.legend()
plt.show()
trues.shape

(1952, 48, 1)

In [8]:
plt.figure()
plt.plot(trues[:,0,-1].reshape(-1), label='GroundTruth')
plt.plot(preds[:,0,-1].reshape(-1), label='Prediction')
plt.legend()
plt.show()

In [13]:
preds_MS = np.load('./results/' + "informer_custom_ftMS_sl96_ll48_pl48_dm256_nh4_el2_dl1_df256_atprob_fc5_ebtimeF_dtTrue_exp" + '/pred.npy')

plt.figure()
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.plot(trues[8000:,0,-1].reshape(-1), label='GroundTruth')
plt.plot(preds[8000:,0,-1].reshape(-1), label='Prediction')
plt.plot(preds_MS[8000:,0,-1].reshape(-1), label='Prediction_MS')


plt.yticks(size=16)
plt.xticks(size=16)
plt.xlabel('number',fontdict={'size':16})
plt.ylabel('data',fontdict={'size':16})
plt.legend(frameon=False)
plt.show()